In [1]:
import openai
import os
from openai import AzureOpenAI
import json
import pandas as pd
from dotenv import load_dotenv
load_dotenv("credentials.env", override=True)

True

Pre-req - create vector index in Azure AI search with the name as "elonmusk-vindex-6whqjwtaeogci"

Use document Elon_Musk.pdf from data folder.

In [2]:
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
deployment_name = "gpt-4o"
aoai_api_version = os.environ["AZURE_OPENAI_API_VERSION"] #Using latest version as of this date. Older versions may not work with the code as syntax has changed drastically.

# Azure AI Search setup
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT"); # Add your Azure AI Search endpoint here
search_key = os.getenv("AZURE_SEARCH_KEY"); # Add your Azure AI Search admin key here
search_index_name = "elonmusk-vindex-6whqjwtaeogci"; # Add your Azure AI Search index name here

In [4]:
# Create an Azure OpenAI client.  
client = openai.AzureOpenAI(        
    #base_url=f"{aoai_endpoint}/openai/deployments/{deployment_name}/extensions/",
    azure_endpoint=aoai_endpoint,    
    api_key=aoai_api_key,    
    api_version=aoai_api_version
)


In [5]:
# Create and return a new chat completion request
    # Be sure to include the "extra_body" parameter to use Azure AI Search as the data source
    #this is Azure Open AI On your data feature

def ask_llm_rag(user_prompt):
    sys_prompt = """You are AI assistant who provides information from retrieved data. If the required information is not available in the retrieved data, 
                apologize and say that the requested information is not available in the retrieved data. You are friendly and concise. 
                Do not provide answers to questions not related to retrieved data or prompt message.
                Provide your response only based on the document provided in the data source. You must generate citation based on the retrieved documents in the response
                Summarize and explain in 1-2 paragraphs with bullet points"""
    messages = [{"role": "system", "content": sys_prompt}]
    messages.append({"role": "user", "content": user_prompt})
    response = client.chat.completions.create(
        model=deployment_name,    
        messages = messages,
        stream=False,
        max_tokens=4000,
        extra_body={
            "data_sources": [
                {
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": search_endpoint,                    
                        "index_name": search_index_name,
                        "authentication": {
                            "type": "api_key",
                            "key": search_key 
                        },                        
                        "top_n_documents": 4,
                        "fields_mapping": {
                            "content_fields_separator": "\\n",                           
                            "filepath_field": "name",
                            "title_field": "title"                            
                        }
                                        
                    }
                }
            ]
        }
    ) 
    return response   

In [6]:
user_prompt = "where did Elon Musk studied"
#"What are the major accomplishments of Elon Musk?" #"What and how many companies and subsidieries founded by Elon Musk?"

response = ask_llm_rag(user_prompt)
print(response.model)
print(response.usage)
print(response.created)
#response.object
#response.choices[0].message.context
from IPython.display import Markdown
Markdown(response.choices[0].message.content)


gpt-4o
CompletionUsage(completion_tokens=110, prompt_tokens=3715, total_tokens=3825, completion_tokens_details=None, prompt_tokens_details=None)
1739717685


Elon Musk attended Queen's University in Kingston, Ontario, starting in 1989 [doc1][doc4]. Two years later, he transferred to the University of Pennsylvania, where he studied until 1995 [doc1]. At the University of Pennsylvania, he earned a Bachelor of Arts in physics and a Bachelor of Science in economics, although the degrees were officially awarded in 1997 [doc1].

List all the citations

In [7]:
response.choices[0].message.context['citations']
#response.choices[0].message.context['citations'][0]
# for citation in response.choices[0].message.context['citations']:
#     print(citation['chunk_id'] + "" if(pd.isnull(citation['filepath'])) else + "-" + citation['filepath'])

[{'content': "aHR0cHM6Ly82d2hxand0YWVvZ2Npc2EuYmxvYi5jb3JlLndpbmRvd3MubmV0L2Vsb25tdXNrLXdpa2kvRWxvbl9NdXNrLnBkZg2\\nQueen's University in Kingston,\nOntario.[40][41] Two years later, he transferred to the University of Pennsylvania, where he studied until\n1995.[42] Although Musk has said that he earned his degrees in 1995, the University of Pennsylvania did\nnot award them until 1997 – a Bachelor of Arts in physics and a Bachelor of Science in economics from\n\nEducation\n\n  \n\nhttps://en.wikipedia.org/wiki/File:PBHS-facade.jpg\nhttps://en.wikipedia.org/wiki/File:PBHS-facade.jpg",
  'title': 'Elon Musk - Wikipedia',
  'url': None,
  'filepath': 'Elon_Musk.pdf',
  'chunk_id': '6'},
 {'content': 'aHR0cHM6Ly82d2hxand0YWVvZ2Npc2EuYmxvYi5jb3JlLndpbmRvd3MubmV0L2Vsb25tdXNrLXdpa2kvRWxvbl9NdXNrLnBkZg2\\n350. Werber, Cassie. "Elon Musk says tax-free carbon is "the dumbest experiment in history" " (ht\ntps://web.archive.org/web/20200826161408/https://qz.com/564782/elon-musk-says-tax-free\n-car

Check what is the intent created 

In [8]:
response.choices[0].message.context['intent']

'["Elon Musk education history", "Where did Elon Musk go to school", "Elon Musk university and college"]'

In [9]:
user_prompt = "What and how many companies and subsidieries founded by Elon Musk"
#"What are the major accomplishments of Elon Musk?" #"What and how many companies and subsidieries founded by Elon Musk?"

response = ask_llm_rag(user_prompt)
print(response.model)
print(response.usage)
print(response.created)
#response.object
#response.choices[0].message.context
from IPython.display import Markdown
Markdown(response.choices[0].message.content)

gpt-4o
CompletionUsage(completion_tokens=308, prompt_tokens=3663, total_tokens=3971, completion_tokens_details=None, prompt_tokens_details=None)
1739717716


Elon Musk has founded several companies and subsidiaries over the years. Here are the key ones:

- **Zip2**: Co-founded with his brother Kimbal Musk, Zip2 was a software company that provided business directories and maps for newspapers [doc2].
- **X.com**: An online payment company that later became PayPal after a merger with Confinity .
- **SpaceX**: Founded in May 2002, SpaceX is a private aerospace manufacturer and space transportation company. Musk serves as the CEO and chief engineer [doc3].
- **Tesla, Inc.**: Although not the original founder, Musk joined Tesla Motors (now Tesla, Inc.) shortly after its founding and became its largest investor, chairman, and later CEO .
- **SolarCity**: Co-founded with his cousins Lyndon and Peter Rive, SolarCity is a solar energy services company .
- **OpenAI**: Co-founded as a non-profit artificial intelligence research company [doc4].
- **Neuralink**: Co-founded to develop implantable brain–machine interfaces .
- **The Boring Company**: Founded to construct tunnels and develop infrastructure for reducing traffic .

These companies span various industries, including software, finance, aerospace, automotive, energy, artificial intelligence, neurotechnology, and infrastructure.

In [10]:
user_prompt = "What are the major accomplishments of Elon Musk?" 

response = ask_llm_rag(user_prompt)
print(response.model)
print(response.usage)
print(response.created)
#response.object
#response.choices[0].message.context
from IPython.display import Markdown
Markdown(response.choices[0].message.content)

gpt-4o
CompletionUsage(completion_tokens=318, prompt_tokens=3760, total_tokens=4078, completion_tokens_details=None, prompt_tokens_details=None)
1739717727


Elon Musk has achieved several notable accomplishments throughout his career:

- **SpaceX**: Musk founded SpaceX in 2002 with the goal of reducing space transportation costs and enabling the colonization of Mars. SpaceX has developed the Falcon and Starship rockets, and it became the first private company to send a spacecraft to the International Space Station (ISS) [doc1].

- **Tesla, Inc.**: As the CEO and product architect of Tesla, Inc., Musk has been instrumental in advancing electric vehicle technology. Tesla's Model S, Model 3, Model X, and Model Y have been significant in popularizing electric cars .

- **SolarCity**: Musk co-founded SolarCity, a solar energy services company, which was later acquired by Tesla and became Tesla Energy. This venture aims to provide sustainable energy solutions .

- **Neuralink**: Musk founded Neuralink, a neurotechnology company focused on developing brain-computer interface technology .

- **The Boring Company**: This infrastructure and tunnel construction services company was founded by Musk to reduce traffic in urban areas through the creation of underground transportation systems .

- **OpenAI**: Musk co-founded OpenAI, a research organization dedicated to ensuring that artificial general intelligence benefits all of humanity .

These accomplishments highlight Musk's influence in various industries, including space exploration, electric vehicles, renewable energy, neurotechnology, and artificial intelligence.

In [12]:
user_prompt = "summarize business career of Elon Musk?" 

response = ask_llm_rag(user_prompt)
print(response.model)
print(response.usage)
print(response.created)
#response.object
#response.choices[0].message.context
from IPython.display import Markdown
Markdown(response.choices[0].message.content)

gpt-4o
CompletionUsage(completion_tokens=298, prompt_tokens=3690, total_tokens=3988, completion_tokens_details=None, prompt_tokens_details=None)
1739717760


Elon Musk's business career is marked by several significant ventures and achievements:

- **Zip2**: Musk's first major business venture was Zip2, a company that provided online city guide software to newspapers. This company was eventually sold to Compaq for nearly $300 million [doc1].
- **X.com and PayPal**: Musk founded X.com, an online payment company, which later became PayPal after a merger. PayPal was sold to eBay for $1.5 billion in stock [doc1].
- **Tesla, Inc.**: Musk joined Tesla Motors, Inc. (now Tesla, Inc.) in its early stages and became its CEO and product architect. Tesla is known for its electric vehicles and renewable energy products .
- **SpaceX**: Musk founded SpaceX with the goal of reducing space transportation costs to enable the colonization of Mars. SpaceX has achieved numerous milestones, including the first privately funded spacecraft to reach the International Space Station .
- **Other Ventures**: Musk has also been involved in several other ventures, including SolarCity (a solar energy services company), Neuralink (a neurotechnology company), and The Boring Company (focused on tunnel construction) .

These ventures highlight Musk's diverse interests and his impact on various industries, from online payments to space exploration and renewable energy.

In [13]:
user_prompt = "list all relatives of Elon Musk?" 

response = ask_llm_rag(user_prompt)
print(response.model)
print(response.usage)
print(response.created)
#response.object
#response.choices[0].message.context
from IPython.display import Markdown
Markdown(response.choices[0].message.content)

gpt-4o
CompletionUsage(completion_tokens=143, prompt_tokens=3745, total_tokens=3888, completion_tokens_details=None, prompt_tokens_details=None)
1739717769


Elon Musk has several relatives, including:

- **Parents**:
  - Errol Musk (father) [doc1]
  - Maye Musk (mother) [doc1]

- **Siblings**:
  - Kimbal Musk (brother) [doc1]
  - Tosca Musk (sister) [doc1][doc4]

- **Half-siblings**:
  - Four paternal half-siblings [doc4]

- **Cousin**:
  - Lyndon Rive [doc1]

These are the relatives mentioned in the retrieved documents.

In [14]:
user_prompt = "list all peoples associated with Elon Musk with their relation to him" 

response = ask_llm_rag(user_prompt)
print(response.model)
print(response.usage)
print(response.created)
#response.object
#response.choices[0].message.context
from IPython.display import Markdown
Markdown(response.choices[0].message.content)

gpt-4o
CompletionUsage(completion_tokens=154, prompt_tokens=3685, total_tokens=3839, completion_tokens_details=None, prompt_tokens_details=None)
1739718318


The retrieved documents mention the following people associated with Elon Musk and their relations to him:

- **Kimbal Musk**: Elon Musk's brother. They co-founded Global Link Information Network, later renamed Zip2, in 1995 [doc4].
- **Greg Kouri**: A business associate who co-founded Global Link Information Network (Zip2) with Elon and Kimbal Musk in 1995 [doc4].
- **Yuri Milner**: Mentioned in the context of an article about Elon Musk in Time magazine [doc1].

The requested information is not available in the retrieved data. Please try another query or topic.

In [127]:
user_prompt = "provide 10 points summary of Musk's life?" 

response = ask_llm_rag(user_prompt)
print(response.model)
print(response.usage)
print(response.created)
#response.object
#response.choices[0].message.context
from IPython.display import Markdown
Markdown(response.choices[0].message.content)

gpt-4o
CompletionUsage(completion_tokens=39, prompt_tokens=3675, total_tokens=3714, completion_tokens_details=None, prompt_tokens_details=None)
1735938455


The requested information is not available in the retrieved data. Please try another query or topic.

In [132]:
user_prompt = "Describe the childhood of Elon Musk based on retrieved data" 

response = ask_llm_rag(user_prompt)
print(response.model)
print(response.usage)
print(response.created)
#response.object
#response.choices[0].message.context
from IPython.display import Markdown
Markdown(response.choices[0].message.content)

gpt-4o
CompletionUsage(completion_tokens=122, prompt_tokens=3719, total_tokens=3841, completion_tokens_details=None, prompt_tokens_details=None)
1735939469


Elon Musk had a turbulent childhood marked by challenges and difficulties. He experienced a turbulent upbringing with an abusive father, which left him scarred [doc1]. Elon Musk has a younger brother, Kimbal, and a younger sister, Tosca, as well as four paternal half-siblings [doc2]. His father was involved in various professions, including being a sailor, consultant, emerald dealer, and property developer, and he partly owned a rental lodge at the Timbavati Private Nature Reserve [doc2].

In [131]:
response.choices[0].message.context['citations']

[{'content': 'aHR0cHM6Ly82d2hxand0YWVvZ2Npc2EuYmxvYi5jb3JlLndpbmRvd3MubmV0L2Vsb25tdXNrLXdpa2kvRWxvbl9NdXNrLnBkZg2\\nhttps://web.archive.org/web/20221006104534/https://www.mercurynews.com/2012/08/13/greg-kouri-early-investor-in-paypal-dies-in-new-york-2/\nhttps://web.archive.org/web/20221006104534/https://www.mercurynews.com/2012/08/13/greg-kouri-early-investor-in-paypal-dies-in-new-york-2/\nhttps://www.cnbc.com/2018/06/19/how-elon-musk-founded-zip2-with-his-brother-kimbal.html',
  'title': 'Elon Musk - Wikipedia',
  'url': None,
  'filepath': 'Elon_Musk.pdf',
  'chunk_id': '3'},
 {'content': 'aHR0cHM6Ly82d2hxand0YWVvZ2Npc2EuYmxvYi5jb3JlLndpbmRvd3MubmV0L2Vsb25tdXNrLXdpa2kvRWxvbl9NdXNrLnBkZg2\\nWalter Isaacson says the billionaire\'s turbulent\nchildhood with an abusive father left him scarred" (https://fortune.com/well/2023/09/17/does-\nelon-musk-have-ptsd-walter-isaacson-biography/). Archived (https://web.archive.org/web/20\n240615085723/https://fortune.com/well/2023/09/17/does-elon-mu